# Setup

In [20]:
# Image from Drive
from google.colab import drive      
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!pip install --quiet -U --no-cache-dir gdown --pre
!pip install --quiet nlp-id
!pip install --quiet emoji
!pip install --quiet imblearn
!pip install --quiet gensim

In [22]:
from nlp_id.stopword import StopWord
from nlp_id.postag import PosTag
from nlp_id.lemmatizer import Lemmatizer

import emoji
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv')
dataset = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/re_dataset.csv', encoding='latin-1')

test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/test_data.csv', encoding='latin-1')
synthetic_test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/synthetic_test_data_5x.csv', encoding='latin-1')
# train data
remaining_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/remaining_data.csv', encoding='latin-1')
eda_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/eda_sample.csv', encoding='latin-1')

kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/kamusalay.csv', encoding='latin-1')
potensi_bias = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/potensi_bias.csv', encoding='latin-1')
most_freq = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/most_freq_words.csv', encoding='latin-1')

# Data Cleaning

In [24]:
# Tokenization & Emoticon Processor

regex_str = []
regex_str.append(r'<[^>]+>') # HTML tags
regex_str.append(r'(?:@[\w_]+)') # @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(sentence):
    return [emoji.demojize(term) for term in tokens_re.findall(sentence)]

In [25]:
# Stopwords Removal & Lemmatizing

lemmatizer = Lemmatizer()
stopword = StopWord()

punctuation = list(string.punctuation)
stops = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“', 'user', 'url',
                                                  'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 
                                                  'x98', 'x84', 'xf', 'xe2', 'x80', 'x94', 'x82', 'xa4', 'x9d', 'xc2',
                                                  'x86', 'x85', 'x81', 'xa6', 'xa3', 'xad', 'x91', 'x81', 'xe3', 'x99',
                                                  'x', 'a', 'n', 'b',
                                                  '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

def remove_stops_and_lemmatize(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word.lower() in kamus_alay.Alay.values:
      row = kamus_alay[kamus_alay['Alay'] == word.lower()]
      word = row['Tidak Alay'].values[0]
    if word.lower() not in stops:
      processed_words.append(lemmatizer.lemmatize(word))
  return processed_words

In [26]:
def remove_stops_and_lemmatize_keep_exclamation(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word == "!":
      processed_words.append(word)
    if word.lower() not in stops:
      processed_words.append(lemmatizer.lemmatize(word))
  return processed_words

# Selected Features

In [27]:
HS_features = [
    'cebong',
    '2019gantipresiden',
    'Exclamation',      # Is there an exclammation?
    'jokowi',
    'prabowo',
    'ahok',
    'lengser',
    'Length',           # Length of tweet
    'Abusive',          # Is there any word that is in the abusive lexicon?
    'budaya',
    'tolol',
    'goblok',
    'Uppercase',        # How many uppercase letters?
    'bubar',
    'dungu',
    'Words',            # Amount of words in tweet
    'kampret',
    'bani',
    'dasar',
    'kafir',
    'otak',
    'katolik',
    'tipu',
    'bom',
    'biadab',
    'ekonomi',
    'kristen',
    'pki',
    'pilkada',
    'sontoloyo'
    ]

HS_word_features = [
    'cebong',
    '2019gantipresiden',
    'jokowi',
    'prabowo',
    'ahok',
    'lengser',
    'budaya',
    'tolol',
    'goblok',
    'bubar',
    'dungu',
    'kampret',
    'bani',
    'dasar',
    'kafir',
    'otak',
    'katolik',
    'tipu',
    'bom',
    'biadab',
    'ekonomi',
    'kristen',
    'pki',
    'pilkada',
    'sontoloyo'
    ]

In [28]:
A_features = [
    'Abusive',          # Is there any word that is in the abusive lexicon?
    'cebong',
    'Length',           # Length of tweet
    'Words',            # Amount of words in tweet
    'goblok',
    'kampret',
    'tolol',
    'anjir',
    'kontol',
    'bajing',
    'anjing',
    'gubernur',
    'dungu',
    'memek',
    'jokowi',
    'otak',
    'kristen',
    'banci',
    'kunyuk',
    'ngentot',
    'bodoh',
    'presiden',
    'asing',
    'setan',
    'babi',
    'bani',
    'budaya',
    'berengsek',
    'bacot',
    'bego'
    ]

A_word_features = [
    'cebong',
    'goblok',
    'kampret',
    'tolol',
    'anjir',
    'kontol',
    'bajing',
    'anjing',
    'gubernur',
    'dungu',
    'memek',
    'jokowi',
    'otak',
    'kristen',
    'banci',
    'kunyuk',
    'ngentot',
    'bodoh',
    'presiden',
    'asing',
    'setan',
    'babi',
    'bani',
    'budaya',
    'berengsek',
    'bacot',
    'bego'
    ]


In [29]:
features = list(set(HS_features + A_features))

In [30]:
word_features  = list(set(HS_word_features + A_word_features))
word_features

['gubernur',
 'ekonomi',
 'bajing',
 'setan',
 'kristen',
 'tolol',
 'bom',
 'berengsek',
 'bacot',
 'kunyuk',
 'otak',
 'tipu',
 'cebong',
 'kampret',
 'pilkada',
 'sontoloyo',
 '2019gantipresiden',
 'dasar',
 'kafir',
 'goblok',
 'pki',
 'dungu',
 'jokowi',
 'babi',
 'budaya',
 'anjir',
 'memek',
 'asing',
 'ngentot',
 'presiden',
 'bodoh',
 'bani',
 'anjing',
 'bego',
 'banci',
 'biadab',
 'lengser',
 'kontol',
 'bubar',
 'ahok',
 'prabowo',
 'katolik']

# Feature Extraction

## Prepare DF

### Train

In [31]:
train_data = pd.concat([
    remaining_data,
    eda_data])

In [32]:
train_tweet = train_data["Tweet"]
train_placeholder = [0 for x in range(len(train_tweet))]
train_df = pd.DataFrame(train_tweet, columns=["Tweet"])

for feature in features:
  train_df[feature] = train_placeholder

### Test

In [33]:
test_tweet = test_data["Tweet"]
test_placeholder = [0 for x in range(len(test_tweet))]
test_df = pd.DataFrame(test_tweet, columns=["Tweet"])

for feature in features:
  test_df[feature] = test_placeholder

### Synthetic Test

In [34]:
syntest_tweet = synthetic_test_data["Tweet"]
syntest_placeholder = [0 for x in range(len(synthetic_test_data))]
syntest_df = pd.DataFrame(syntest_tweet, columns=["Tweet"])

for feature in features:
  syntest_df[feature] = syntest_placeholder

## Exclamation

In [35]:
def EF_exclamation(data):
  for i, row in data.iterrows():
    words = remove_stops_and_lemmatize_keep_exclamation(tokenize(row["Tweet"]))

    for word in words:
      if word == "!":
        data.loc[i, 'Exclamation'] = 1
        break

  return data

In [36]:
train_df = EF_exclamation(train_df)
test_df = EF_exclamation(test_df)
syntest_df = EF_exclamation(syntest_df)

## Length

In [37]:
def EF_Length(df):
  for i, row in df.iterrows():
    words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
    length = len(row["Tweet"])  # jumlah karakter dalam tweet
    df.loc[i, 'Length'] = length
  
  return df

In [38]:
train_df = EF_Length(train_df)
test_df = EF_Length(test_df)
syntest_df = EF_Length(syntest_df)

## Words

In [39]:
def EF_Words(df):
  for i, row in df.iterrows():
    words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
    length = len(words)   # jumlah kata dalam tweet
    df.loc[i, 'Words'] = length
  
  return df

In [40]:
train_df = EF_Words(train_df)
test_df = EF_Words(test_df)
syntest_df = EF_Words(syntest_df)

## Uppercase

In [41]:
def EF_Uppercase(df):
  for i, row in df.iterrows():
    uppercase = sum(1 for c in row["Tweet"] if c.isupper())
    df.loc[i, 'Uppercase'] = uppercase
  
  return df

In [42]:
def EF_Uppercase_2(df, sdf):
  for i, row in df.iterrows():
    uppercase = sum(1 for c in row["Tweet"] if c.isupper())
    df.loc[i, 'Uppercase'] = uppercase
    sdf.loc[i*5, 'Uppercase'] = uppercase
    sdf.loc[i*5 +1, 'Uppercase'] = uppercase
    sdf.loc[i*5 +2, 'Uppercase'] = uppercase
    sdf.loc[i*5 +3, 'Uppercase'] = uppercase
    sdf.loc[i*5 +4, 'Uppercase'] = uppercase
  
  return df

In [43]:
train_df = EF_Uppercase(train_df)
test_df = EF_Uppercase_2(test_df, syntest_df)

## Abusive

In [44]:
def EF_abusive(data):
  for i, row in data.iterrows():
    words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))

    for word in words:
      if word.lower() in abusive_lexicon.values:
        data.loc[i, 'Abusive'] = 1
        break

  return data

In [45]:
train_df = EF_abusive(train_df)
test_df = EF_abusive(test_df)
syntest_df = EF_abusive(syntest_df)

## Word Features

In [46]:
def extract_by_freq_word(data):
  for i, row in data.iterrows():
    words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))

    for word in words:
      if word.lower() in word_features:
        data.loc[i, word.lower()] = 1

  return data

In [47]:
train_df = extract_by_freq_word(train_df)
test_df = extract_by_freq_word(test_df)
syntest_df = extract_by_freq_word(syntest_df)

# Store Dataframe

In [48]:
train_df["HS"] = train_data["HS"]
train_df["A"] = train_data["Abusive"]

test_df["HS"] = test_data["HS"]
test_df["A"] = test_data["Abusive"]

syntest_df["HS"] = synthetic_test_data["HS"]
syntest_df["A"] = synthetic_test_data["Abusive"]

In [49]:
train_df

Tweet  gubernur  ekonomi  \
0     - disaat semua cowok berusaha melacak perhatia...         0        0   
1     Nah admin belanja satu lagi port terbaik nak m...         0        0   
2     Aktor huruhara 98 Prabowo S ingin lengserkan p...         0        0   
3     USER Bu guru enakan jadi jablay atau guru esde...         0        0   
4     USER USER USER USER USER USER Lawan bicara gw ...         0        0   
...                                                 ...       ...      ...   
3287  USER keluarga halilintar hmm masa km gatau. udik'         0        0   
3288  #2019GantiPresiden scra konstitusional hak set...         0        1   
3289  USER si ahok pantasnya dinobatkan sbgai Babi n...         0        0   
3290  YA ALLAH INI XL GAK KURANG KAMPRET LAGI APA KO...         0        0   
3291  USER Den...den...\nNista banget hidup lo..\nUd...         0        0   

      bajing  setan  kristen  tolol  bom  Length  berengsek  ...  banci  \
0          0      0        0      0    0      74          0  ...      0   
1          0      0        0      0    0      77          0  ...      0   
2          0      0        0      0    0     135          0  ...      0   
3          0      0        0      0    0     133          0  ...      0   
4          0      0        0      0    0     174          0  ...      0   
...      ...    ...      ...    ...  ...     ...        ...  ...    ...   
3287       0      0        0      0    0      49          0  ...      0   
3288       0      0        0      0    0     118          0  ...      0   
3289       0      0        0      0    0      52          0  ...      0   
3290       0      0        0      0    0      61          0  ...      0   
3291       0      0        0      0    0     232          0  ...      0   

      biadab  lengser  kontol  bubar  ahok  prabowo  katolik  HS  A  
0          0        0       0      0     0        0        0   1  1  
1          0        0       0      0     0        0        0   0  0  
2          0        1       0      0     0        1        0   1  0  
3          0        0       0      0     0        0        0   1  1  
4          0        0       0      0     0        0        0   1  1  
...      ...      ...     ...    ...   ...      ...      ...  .. ..  
3287       0        0       0      1     0        0        0   1  1  
3288       0        0       0      0     0        0        0   1  1  
3289       0        0       0      0     1        0        0   1  1  
3290       0        0       0      0     0        0        0   1  1  
3291       0        0       0      0     0        0        0   1  1  

[9219 rows x 50 columns]

In [50]:
test_df

Tweet  gubernur  ekonomi  \
0     USER Ooo camtu.. Biasalah kena seimbang.. Ada ...         0        0   
1     Presiden Minta Sistem Perizinan Berusaha Terin...         0        0   
2     ð???ð???ð???tak berambisi? Mimin bisa aja judu...         0        0   
3     USER Kemaren gubernur pilihan umat impor bawan...         1        0   
4     4. Apakah pernah mengalami kendala dalam menga...         0        0   
...                                                 ...       ...      ...   
3945  Doi tuh native speaker ya.\n\nMother language-...         0        0   
3946  RT USER Kasian ya kacung tetap kacung anies sandi         0        0   
3947  USER kacrut , rank 10 tapi mainnya gk cerdas a...         0        0   
3948  USER Klau kmu di banding AR jauhlah.koen menun...         0        0   
3949  Mana para babi ahok yg ngejek onta, biar di in...         0        0   

      bajing  setan  kristen  tolol  bom  Length  berengsek  ...  banci  \
0          0      0        0      0    0     276          0  ...      0   
1          0      0        0      0    0     164          0  ...      0   
2          0      0        0      0    0      53          0  ...      0   
3          0      0        0      0    0      93          0  ...      0   
4          0      0        0      0    0      69          0  ...      0   
...      ...    ...      ...    ...  ...     ...        ...  ...    ...   
3945       0      0        0      0    0      88          0  ...      0   
3946       0      0        0      0    0      49          0  ...      0   
3947       0      0        0      0    0      62          0  ...      0   
3948       0      0        0      0    0     219          0  ...      0   
3949       0      0        0      0    0      59          0  ...      0   

      biadab  lengser  kontol  bubar  ahok  prabowo  katolik  HS  A  
0          0        0       0      0     0        0        0   0  0  
1          0        0       0      0     0        0        0   0  0  
2          0        0       0      0     0        0        0   0  0  
3          0        0       0      0     0        0        0   0  0  
4          0        0       0      0     0        0        0   0  0  
...      ...      ...     ...    ...   ...      ...      ...  .. ..  
3945       0        0       0      0     0        0        0   1  1  
3946       0        0       0      0     0        0        0   1  1  
3947       0        0       0      0     0        0        0   1  1  
3948       0        0       0      0     0        0        0   1  1  
3949       0        0       0      0     1        0        0   1  1  

[3950 rows x 50 columns]

In [51]:
syntest_df

Tweet  gubernur  ekonomi  \
0      user ooo camtu.. biasalah kena seimbang.. ada ...         0        0   
1      user ooo camtu.. biasalah kena seimbang.. ada ...         0        0   
2      user ooo camtu.. biasalah kena seimbang.. ada ...         0        0   
3      user ooo camtu.. biasalah kena seimbang.. ada ...         0        0   
4      user ooo camtu.. biasalah kena seimbang.. ada ...         0        0   
...                                                  ...       ...      ...   
19745  mana para babi adi yg ngejek onta, biar di inj...         0        0   
19746  mana para babi doni yg ngejek onta, biar di in...         0        0   
19747  mana para babi zaidan yg ngejek onta, biar di ...         0        0   
19748  mana para babi baraz yg ngejek onta, biar di i...         0        0   
19749  mana para babi usman yg ngejek onta, biar di i...         0        0   

       bajing  setan  kristen  tolol  bom  Length  berengsek  ...  banci  \
0           0      0        0      0    0     276          0  ...      0   
1           0      0        0      0    0     276          0  ...      0   
2           0      0        0      0    0     276          0  ...      0   
3           0      0        0      0    0     276          0  ...      0   
4           0      0        0      0    0     276          0  ...      0   
...       ...    ...      ...    ...  ...     ...        ...  ...    ...   
19745       0      0        0      0    0      58          0  ...      0   
19746       0      0        0      0    0      59          0  ...      0   
19747       0      0        0      0    0      61          0  ...      0   
19748       0      0        0      0    0      60          0  ...      0   
19749       0      0        0      0    0      60          0  ...      0   

       biadab  lengser  kontol  bubar  ahok  prabowo  katolik  HS  A  
0           0        0       0      0     0        0        0   0  0  
1           0        0       0      0     0        0        0   0  0  
2           0        0       0      0     0        0        0   0  0  
3           0        0       0      0     0        0        0   0  0  
4           0        0       0      0     0        0        0   0  0  
...       ...      ...     ...    ...   ...      ...      ...  .. ..  
19745       0        0       0      0     0        0        0   1  1  
19746       0        0       0      0     0        0        0   1  1  
19747       0        0       0      0     0        0        0   1  1  
19748       0        0       0      0     0        0        0   1  1  
19749       0        0       0      0     0        0        0   1  1  

[19750 rows x 50 columns]

In [52]:
syntest_df.prabowo.sum()

0

In [53]:
from google.colab import files

train_df.to_csv('extracted_train_data.csv', encoding = 'latin-1') 
files.download('extracted_train_data.csv')

test_df.to_csv('extracted_test_data.csv', encoding = 'latin-1') 
files.download('extracted_test_data.csv')

syntest_df.to_csv('extracted_synthetic_test_data.csv', encoding = 'latin-1') 
files.download('extracted_synthetic_test_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>